In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *
from setting import *

## Run Kallisto to quantify transcripts and make transcript-x-sample and gene-x-sample

In [ ]:
# paired_fastq_gz_file_paths = tuple((
#     '../data/{}_1.fq.gz'.format(sample_id),
#     '../data/{}_2.fq.gz'.format(sample_id),
# ) for sample_id in (
#     'NC1',
#     'NC2',
#     'NC3',
#     'NM1',
#     'NM2',
#     'NM3',
#     'WC1',
#     'WC2',
#     'WC3',
#     'WM1',
#     'WM2',
#     'WM3',
# ))

# for fastq_gz_file_paths in paired_fastq_gz_file_paths:

#     for fastq_gz_file_path in fastq_gz_file_paths:

#         assert os.path.isfile(fastq_gz_file_path), fastq_gz_file_path

In [ ]:
# cdna_fasta_gz_file_path = '../../../data/grch/Homo_sapiens.GRCh38.cdna.all.fa.gz'

# assert os.path.isfile(cdna_fasta_gz_file_path), cdna_fasta_gz_file_path

In [ ]:
# ccal.establish_path(
#     '../output/kallisto',
#     'directory',
# )

# tpms = []

# for paired_fastq_gz_file_path_0, paired_fastq_gz_file_path_1 in paired_fastq_gz_file_paths:

#     sample_id = os.path.commonprefix((
#         paired_fastq_gz_file_path_0.split(sep='/')[-1],
#         paired_fastq_gz_file_path_1.split(sep='/')[-1],
#     )).strip('_.')
    
#     print(sample_id)

#     abundance_file_path = '../output/kallisto/{}/abundance.tsv'.format(sample_id)

#     if not os.path.isfile(abundance_file_path):

#         ccal.count_transcripts_using_kallisto_quant(
#             (
#                 paired_fastq_gz_file_path_0,
#                 paired_fastq_gz_file_path_1,
#             ),
#             cdna_fasta_gz_file_path,
#             '../output/kallisto/{}'.format(sample_id),
#             n_job=a.N_JOB,
#         )

#     tpm = pd.read_table(
#         abundance_file_path,
#         index_col=0,
#     )['tpm']
    
#     tpm.name = '{}_tpm'.format(sample_id)
    
#     tpms.append(tpm)
    
# enst_x_sample = pd.concat(
#     tpms,
#     axis=1,
# )

# enst_x_sample

In [ ]:
# gene_x_sample = enst_x_sample.copy()

# enst = pd.read_table(
#     '../../../data/enst.tsv',
#     index_col=1,
# )

# enst_gene_name = enst['Gene name'].to_dict()

# gene_x_sample.index = enst_x_sample.index.map(enst_gene_name.get)

# print(gene_x_sample.shape)

# gene_x_sample = gene_x_sample.loc[~gene_x_sample.index.isnull()]

## Download gene-x-sample

In [ ]:
gene_x_sample = ccal.download_and_parse_geo_data(
    'GSE48766',
    directory_path='../data'
)['gene_x_sample']

gene_x_sample.index = gene_x_sample.index.str.upper()

## Keep only the maximum signal for duplicated genes and write gene-x-sample to a file

In [ ]:
print(gene_x_sample.shape)

max_tpm__gene_x_sample = gene_x_sample.groupby(level=0).max()

print(max_tpm__gene_x_sample.shape)

max_tpm__gene_x_sample.sort_index(inplace=True)

max_tpm__gene_x_sample.index.name = 'Gene'

max_tpm__gene_x_sample.columns.name = 'Sample'

max_tpm__gene_x_sample.to_csv(
    '../output/max__gene_x_sample.tsv',
    sep='\t',
)

max_tpm__gene_x_sample

## Process gene-x-sample, look at it, and write it to a file

In [ ]:
processed__gene_x_sample = ccal.read_and_process_feature_x_sample(
    max_tpm__gene_x_sample,
    nanize=0,
    log_base='2',
    plot=True,
)

ccal.plot_distributions(
    processed__gene_x_sample.columns,
    tuple(processed__gene_x_sample[column] for column in processed__gene_x_sample.columns),
    plot_rug=False,
    title='Processed Value Distribution',
    xaxis_title='Processed Value',
)

processed__gene_x_sample.to_csv(
    '../output/processed__gene_x_sample.tsv',
    sep='\t',
)

processed__gene_x_sample